In [32]:
from optlang import Model, Variable, Constraint, Objective
import pandas as pd
from copy import deepcopy

Design parameters

In [2]:
# Load pairwise distance
df_pairdist = pd.read_csv('./output/pfba_pairwise_distance.csv', sep='\t', index_col=0)

# MILP parameters
num_solns = 3 # Number of solutions
num_muts = 3 # Number of mutants

#### Define model

In [3]:
model = Model(name='mfa_design')

#### Define variables

In [4]:
cols = df_pairdist.columns.tolist()
colpairs = []
for i in range(0, len(cols)-1):
    for j in range(i+1, len(cols)):
        colpairs.append([cols[i], cols[j]])

y_list = [Variable('y__' + i, type='binary', lb=0, ub=1) for i in cols]
r_list = [Variable('__'.join(['r'] + i), type='continuous', lb=0, ub=1) for i in colpairs]

In [5]:
vars_model = model.variables.keys()
for var in y_list + r_list:
    if var.name not in vars_model:
        model.add(var)
        
y_list = [i.name for i in model.variables if i.name[:3] == 'y__']
r_list = [i.name for i in model.variables if i.name[:3] == 'r__']

#### Define objective

In [6]:
c_vector = []
for r_var in r_list:
    i,j = r_var.split('__')[1:]
    c_vector.append(df_pairdist.loc[i,j])
    
obj = Objective(sum(c_vector[i] * model.variables[r_list[i]] for i in range(0, len(r_list))))

In [7]:
model.objective = obj

#### Define constraints

In [8]:
# List 1: r_ij <= y_i
cons_list1 = []
for r_var in r_list:
    r_ij = model.variables[r_var]
    y_i = model.variables['y__' + r_var.split('__')[1]]
    cons_list1.append(Constraint(r_ij - y_i, ub=0))
    
# List 2: r_ij <= y_j
cons_list2 = []
for r_var in r_list:
    r_ij = model.variables[r_var]
    y_j = model.variables['y__' + r_var.split('__')[2]]
    cons_list2.append(Constraint(r_ij - y_j, ub=0))
    
# List 3: r_ij >= y_i + y_j - 1
cons_list3 = []
for r_var in r_list:
    r_ij = model.variables[r_var]
    y_i = model.variables['y__' + r_var.split('__')[1]]
    y_j = model.variables['y__' + r_var.split('__')[2]]
    cons_list3.append(Constraint(r_ij - y_i - y_j, lb=-1))
    
# Constraint 4: sum(y) = L
cons_4 = [Constraint(sum([model.variables[i] for i in y_list]), lb=num_muts, ub=num_muts)]

# Constraint 5: y__WT = 1
cons_5 = [Constraint(model.variables['y__WT'], lb=1, ub=1)]

In [9]:
model.add(cons_list1 + cons_list2 + cons_list3 + cons_4 + cons_5)

### Example 1: Solve

In [10]:
cols = ['mutants_set', 'sum_pairwise_dist']
df_solns = pd.DataFrame(columns=cols)

for i in range(0, num_solns):
    
    for soln_iter in range(0, i):
        prev_soln = df_solns.mutants_set[soln_iter].split(',')
        cons = Constraint(sum([model.variables['y__' + val] for val in prev_soln]),
                          lb=0, ub=num_muts-1)
        model.add(cons)
        
    model.optimize()
    
    muts = []
    for var in model.variables.values():
        if var.name[:3] == 'y__' and var.primal == 1:
            muts.append(var.name.split('__')[1])
    df_solns.loc[i, 'mutants_set'] = ','.join(muts)
    df_solns.loc[i, 'sum_pairwise_dist'] = model.objective.value

In [11]:
df_solns

,mutants_set,sum_pairwise_dist
0,"WT,ATPS4r_ko,FBA_ko",3.28948
1,"WT,ATPS4r_ko,FUM_ko",2.89981
2,"WT,ATPS4r_ko,MDH_ko",2.32051


#### Check solutions and debug
Reason for selection of mutant:<br>
ATPS4r: Switch to acetate and formate fermentation<br>
FBA: Switch to PPP (thus higher PPP flux) and ferment formate<br>
FUM: Switch to PPP<br>
MDH: Higher TCA flux, activation of malic enzyme<br>

In [30]:
df_pfba = pd.read_csv('./output/pfba_ecoli_core_filtered2.csv', sep='\t', index_col=0)
df_pfba.loc[:, ['WT', 'ATPS4r_ko', 'FBA_ko', 'FUM_ko', 'MDH_ko']]

### Example 2: Solve with ATP synthase KO exclusion

In [38]:
model_noATPS = deepcopy(model)
model_noATPS.add(Constraint(model_noATPS.variables['y__ATPS4r_ko'], lb=0, ub=0))

In [39]:
cols = ['mutants_set', 'sum_pairwise_dist']
df_solns2 = pd.DataFrame(columns=cols)

for i in range(0, num_solns):
    
    for soln_iter in range(0, i):
        prev_soln = df_solns2.mutants_set[soln_iter].split(',')
        cons = Constraint(sum([model_noATPS.variables['y__' + val] for val in prev_soln]),
                          lb=0, ub=num_muts-1)
        model_noATPS.add(cons)
        
    model_noATPS.optimize()
    
    muts = []
    for var in model_noATPS.variables.values():
        if var.name[:3] == 'y__' and var.primal == 1:
            muts.append(var.name.split('__')[1])
    df_solns2.loc[i, 'mutants_set'] = ','.join(muts)
    df_solns2.loc[i, 'sum_pairwise_dist'] = model_noATPS.objective.value

In [40]:
df_solns2

,mutants_set,sum_pairwise_dist
0,"WT,FBA_ko,MDH_ko",1.89431
1,"WT,FBA_ko,G6PDH2r_ko",1.87973
2,"WT,FBA_ko,TALA_ko",1.85795


#### Check solutions and debug
Reason for selection of mutant:<br>
FBA: Switch to PPP (thus higher PPP flux) and ferment formate<br>
MDH: Higher TCA flux, activation of malic enzyme<br>
G6PDH2r: Higher (but less than MDH KO) TCA flux, higher EMP flux<br>
TALA: Higher (but less than MDH KO) TCA flux, higher EMP flux<br>

In [41]:
df_pfba = pd.read_csv('./output/pfba_ecoli_core_filtered2.csv', sep='\t', index_col=0)
df_pfba.loc[:, ['WT', 'FBA_ko', 'MDH_ko', 'G6PDH2r_ko', 'TALA_ko']]

,WT,FBA_ko,MDH_ko,G6PDH2r_ko,TALA_ko
ACKr,0.000000,0.000000,0.000000,0.000000,0.000000
ACONTa,6.007250,0.759585,8.339762,7.803302,7.635241
ACONTb,6.007250,0.759585,8.339762,7.803302,7.635241
ACt2r,0.000000,0.000000,0.000000,0.000000,0.000000
ADK1,0.000000,4.607429,3.674637,0.000000,0.000000
AKGDH,5.064376,0.000000,7.448786,6.871333,6.702253
ATPM,8.390000,21.113487,8.390000,8.390000,8.390000
ATPS4r,45.514010,63.788053,49.267736,41.353546,41.742849
BIOMASS_Ecoli_core_w_GAM,0.873922,0.704037,0.825819,0.863813,0.864759
CO2t,-22.809833,-26.641181,-24.856847,-23.239993,-23.199742
